# UUI Projekt - Sara Kolac

TEMA: Razvoj sustava za automatizirano vođenje inventara koristeći ontologije: Implementacija sustava koji koristi ontologije za organizaciju i praćenje inventara u skladištima ili maloprodajama.

In [1]:
#!pip install owlready2

In [3]:
from owlready2 import *

onto = get_ontology("file://warehouse_inventory.owl")

with onto:
    #za Artikl
    class Item(Thing): pass
    class Resource(Thing): pass

    #pozicija artikla
    class Warehouse(Resource): pass
    class Section(Resource): pass
    class Shelf(Resource): pass

    #podklase Artikla
    class has_ID(Item >> str): pass
    class has_type(Item >> str): pass
    class has_quantity(Item >> int): pass   
    class on_shelf(Item >> Shelf): pass

    #instance
    warehouse_main = Warehouse("MainWarehouse")
    section_a = Section("SectionA")
    section_b = Section("SectionB")
    shelf_a1 = Shelf("ShelfA1")
    shelf_b1 = Shelf("ShelfB1")
    
    #restrikcije
    section_a.part_of = [warehouse_main]
    section_b.part_of = [warehouse_main]
    shelf_a1.part_of = [section_a]
    shelf_b1.part_of = [section_b]

    #instance item
    item_laptop = Item("Laptop",
                       has_ID=["ID123"],
                       has_type=["electronics"],
                       has_quantity=[20],
                       on_shelf=[shelf_a1])
    item_phone = Item("Phone",
                      has_ID=["ID321"],
                      has_type=["electronics"],
                      has_quantity=[50],
                      on_shelf=[shelf_b1])

#spremanje u datoteku
onto.save("warehouse_inventory.owl")


def new_item(name, item_id, item_type, quantity, shelf_name):
        new_item = onto.Item(name,
                             has_ID=[item_id],
                             has_type=[item_type],
                             has_quantity=[quantity],
                             on_shelf=[shelf_name])
        
        onto.save("warehouse_inventory.owl")
        
        print(f"Added {name} to shelf {shelf_name}")



def move_item(item_name, new_shelf_name):

    item = onto.search_one(iri=f"*{item_name}")
    new_shelf = onto.search_one(iri=f"*{new_shelf_name}")
    
    if item and new_shelf:
        item.on_shelf = [new_shelf]
        print(f"Moved {item_name} to {new_shelf_name}")
    else:
        print(f"Wrong input")


def add_stock(item_name, quantity):
    
    item = onto.search_one(iri=f"*{item_name}")
    if item:
        item.has_quantity[0] += quantity
        print(f"Added {quantity} to {item_name}. New quantity: {item.has_quantity[0]}")
    else:
        print(f"Wrong input")




def remove_stock(item_name, quantity):
    
    item = onto.search_one(iri=f"*{item_name}")
    if item:
        if item.has_quantity[0] >= quantity:
            item.has_quantity[0] -= quantity
            print(f"Removed {quantity} from {item_name}. New quantity: {item.has_quantity[0]}")
        else:
            print(f"Error: Not enough stock of {item_name} to remove {quantity}")
    else:
        print(f"Wrong input")




# ispitujemo 
def query_shelf_for_item(item_name):
    item = onto.search_one(iri=f"*{item_name}")
    if item and item.on_shelf:
        shelf_name = item.on_shelf[0].name
        print(f"{item_name} is located on shelf: {shelf_name}")
    else:
        print(f"Item doesn't exist")



def query_items_on_shelf(shelf_name):
    shelf = onto.search_one(iri=f"*{shelf_name}")
    if shelf:
        print(f"Items on shelf {shelf_name}")
        items = [item for item in onto.Item.instances() if shelf in item.on_shelf]
        if items:
            for item in items:
                print(f"{item.name} (Quantity: {item.has_quantity[0]})")
        else:
            print(f"No items found on shelf '{shelf_name}'")
    else:
        print(f"Shelf does not exist")


#ispisivanje trenutnog inventara
def inventory():
    print(f"Current Inventory:")
    for item in onto.Item.instances():
        print(f"Item: {item.name}")
        print(f"ID: {item.has_ID[0]}")
        print(f"Type: {item.has_type[0]}")
        print(f"Quantity: {item.has_quantity[0]}")
        print(f"Shelf: {item.on_shelf[0].name}")
        print(f"Section: {item.on_shelf[0].part_of[0].name}")
        print(f"Warehouse: {item.on_shelf[0].part_of[0].part_of[0].name}")
        print(f"______")


if __name__ == "__main__":
    inventory()

Current Inventory:
Item: Laptop
ID: ID123
Type: electronics
Quantity: 20
Shelf: ShelfA1
Section: SectionA
Warehouse: MainWarehouse
______
Item: Phone
ID: ID321
Type: electronics
Quantity: 50
Shelf: ShelfB1
Section: SectionB
Warehouse: MainWarehouse
______


In [5]:
query_shelf_for_item("Laptop")

Laptop is located on shelf: ShelfA1


In [7]:
query_items_on_shelf("ShelfA1")

Items on shelf ShelfA1
Laptop (Quantity: 20)


In [9]:
new_item("Tablet", "ID111", "electronics", 15, shelf_a1)

Added Tablet to shelf warehouse_inventory.ShelfA1


In [11]:
inventory()

Current Inventory:
Item: Laptop
ID: ID123
Type: electronics
Quantity: 20
Shelf: ShelfA1
Section: SectionA
Warehouse: MainWarehouse
______
Item: Phone
ID: ID321
Type: electronics
Quantity: 50
Shelf: ShelfB1
Section: SectionB
Warehouse: MainWarehouse
______
Item: Tablet
ID: ID111
Type: electronics
Quantity: 15
Shelf: ShelfA1
Section: SectionA
Warehouse: MainWarehouse
______


In [13]:
remove_stock("Laptop", 5)

Removed 5 from Laptop. New quantity: 15


In [15]:
inventory()

Current Inventory:
Item: Laptop
ID: ID123
Type: electronics
Quantity: 15
Shelf: ShelfA1
Section: SectionA
Warehouse: MainWarehouse
______
Item: Phone
ID: ID321
Type: electronics
Quantity: 50
Shelf: ShelfB1
Section: SectionB
Warehouse: MainWarehouse
______
Item: Tablet
ID: ID111
Type: electronics
Quantity: 15
Shelf: ShelfA1
Section: SectionA
Warehouse: MainWarehouse
______


In [17]:
add_stock("Tablet", 10)

Added 10 to Tablet. New quantity: 25


In [19]:
inventory()

Current Inventory:
Item: Laptop
ID: ID123
Type: electronics
Quantity: 15
Shelf: ShelfA1
Section: SectionA
Warehouse: MainWarehouse
______
Item: Phone
ID: ID321
Type: electronics
Quantity: 50
Shelf: ShelfB1
Section: SectionB
Warehouse: MainWarehouse
______
Item: Tablet
ID: ID111
Type: electronics
Quantity: 25
Shelf: ShelfA1
Section: SectionA
Warehouse: MainWarehouse
______


In [21]:
move_item("Tablet", "ShelfB1")

Moved Tablet to ShelfB1


In [23]:
inventory()

Current Inventory:
Item: Laptop
ID: ID123
Type: electronics
Quantity: 15
Shelf: ShelfA1
Section: SectionA
Warehouse: MainWarehouse
______
Item: Phone
ID: ID321
Type: electronics
Quantity: 50
Shelf: ShelfB1
Section: SectionB
Warehouse: MainWarehouse
______
Item: Tablet
ID: ID111
Type: electronics
Quantity: 25
Shelf: ShelfB1
Section: SectionB
Warehouse: MainWarehouse
______
